In [1]:
import glob
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry

# --- PARAMETERS ---
aperture_radius = 5.0      # in pixels
annulus_r_in = 8.0
annulus_r_out = 12.0
gain = 11.6                 # electrons per ADU (adjust if known)
latitude = 32.7794         # telescope latitude
longitude = 78.9641        # telescope longitude
elevation = 4500           # meters

# --- STAR LIST ---
# CSV with columns: name, ra, dec
stars = pd.read_csv("New_Stars.csv")

paths = [
    "New_Data/NGC1039/*.fits",
    "New_Data/NGC1039_40/*.fits",
    "New_Data/NGC1039_45/*.fits",
    "New_Data/NGC1039_55/*.fits",
    "New_Data/NGC1039_65/*.fits",
    "New_Data/NGC1039_75/*.fits",
    "New_Data/NGC1039_cul/*.fits",
    # "New_Data/NGC1049/*.fits",
]

fits_files = sorted([file for pattern in paths for file in glob.glob(pattern)])

# --- RESULTS STORAGE ---
results = []

for fits_file in fits_files:
    # --- Skip if no stars detected or flux empty ---

    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        w = WCS(header)
        band = header.get("FILTER", "Unknown")
        data = hdul[0].data.astype(float)
        header = hdul[0].header
        w = WCS(header)
    
    # Observation time
    obs_time = header.get("DATE-OBS")
    time_obj = Time(obs_time)
    
    # Telescope location
    location = EarthLocation(lat=latitude*u.deg, lon=longitude*u.deg, height=elevation*u.m)
    
    # --- Convert RA/Dec → pixel coordinates for aperture photometry ---
    skycoords = np.vstack([stars['ra'], stars['dec']]).T
    pix_coords = w.wcs_world2pix(skycoords, 0)


    # --- Debug check ---
    # print("Pixel coordinates:\n", pix_coords)

    # Identify any invalid ones
    invalid = np.isnan(pix_coords).any(axis=1)
    if np.any(invalid):
        print("Warning: Some coordinates could not be mapped to valid pixels!")
        print("Invalid star indices:", np.where(invalid)[0])
        # Optional: drop those invalid stars
        skycoords = skycoords[~invalid]
        pix_coords = pix_coords[~invalid]

    # --- Aperture & annulus ---
    apertures = CircularAperture(pix_coords, r=aperture_radius)
    annulus_apertures = CircularAnnulus(pix_coords, r_in=annulus_r_in, r_out=annulus_r_out)

    # --- Perform aperture photometry ---
    rawflux_table = aperture_photometry(data, apertures)
    bkgflux_table = aperture_photometry(data, annulus_apertures)

    # --- Background correction ---
    bkg_mean = bkgflux_table['aperture_sum'] / annulus_apertures.area
    bkg_sum = bkg_mean * apertures.area
    final_flux = rawflux_table['aperture_sum'] - bkg_sum

    # --- Background RMS from annulus ---
    annulus_masks = annulus_apertures.to_mask(method='center')
    bkg_rms = []
    for mask in annulus_masks:
        annulus_data = mask.multiply(data)
        annulus_data_1d = annulus_data[mask.data > 0]
        bkg_rms.append(np.std(annulus_data_1d))
    bkg_rms = np.array(bkg_rms)

    # --- Uncertainties ---
    N_ap = apertures.area
    flux_err = np.sqrt(final_flux * gain + N_ap * (bkg_rms**2))
    inst_mag = -2.5 * np.log10(final_flux)
    del_mag = 1.0857 * (flux_err / final_flux)

    # Convert RA/Dec → pixel coordinates properly
    ra_vals = np.array(stars['ra'], dtype=float)
    dec_vals = np.array(stars['dec'], dtype=float)
    pix_coords = np.column_stack(w.wcs_world2pix(ra_vals, dec_vals, 0))

    stars_coord = SkyCoord(ra=ra_vals * u.deg, dec=dec_vals * u.deg)

    # --- Compute Alt/Az of stars for this observation ---
    altaz_frame = AltAz(obstime=time_obj, location=location)
    stars_altaz = stars_coord.transform_to(altaz_frame)

    # --- Store results ---
    for i, star in enumerate(stars['name']):
        results.append({
            'file': fits_file,
            'index': fits_file.split('\\')[-1].split('.')[0],
            'filter': band,
            'star': star,
            'ra_deg': stars['ra'][i],
            'dec_deg': stars['dec'][i],
            'alt_deg': stars_altaz.alt[i].value,
            'az_deg': stars_altaz.az[i].value,
            'flux': final_flux[i],
            'flux_err': flux_err[i],
            'inst_mag': inst_mag[i],
            'del_mag': del_mag[i]
        })

# --- SAVE OUTPUT ---
df = pd.DataFrame(results)
df.dropna(inplace=True)
df.to_csv("Cluster_Photometry.csv", index=False)
print("✅ All done! Results saved to Cluster_Photometry.csv")


✅ All done! Results saved to Cluster_Photometry.csv
